### EXTRAÇÃO DOS DADOS 
Processo de conexão e requisição via <i>API</i>, a requisição foi dividia em dois blocos: <hr>

In [4]:
import pandas as pd
import requests

###### 1 - REQUISIÇÃO LISTA DE DEPUTADOS

In [5]:
url_requisicao_deputados = "https://dadosabertos.camara.leg.br/api/v2/deputados"
resultado = requests.get(url_requisicao_deputados)
deputados = pd.DataFrame(resultado.json()['dados'])

lista_exclusao_deputados = ['uri', 'uriPartido', 'idLegislatura', 'urlFoto']
i = 0

while i < len(lista_exclusao_deputados):
    del deputados[lista_exclusao_deputados[i]] 
    i += 1


deputados.nome = [str(nome).upper() for nome in deputados.nome]
deputados.email = [str(email).upper() for email in deputados.email]

deputados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            513 non-null    int64 
 1   nome          513 non-null    object
 2   siglaPartido  513 non-null    object
 3   siglaUf       513 non-null    object
 4   email         513 non-null    object
dtypes: int64(1), object(4)
memory usage: 12.1+ KB


###### 2 - REQUISIÇÃO LISTA DE GASTOS

In [6]:
gastos = []
for id in deputados.id:
    url_despesa = "https://dadosabertos.camara.leg.br/api/v2/deputados/"
    url_despesa = url_despesa +str(id)+"/despesas?ordem=ASC&ano=2020&itens=999999"
    resposta = requests.get(url_despesa)
    gasto = pd.DataFrame(resposta.json()['dados'])
    gasto['id'] = id
    gastos.append(gasto)
    
total_gastos = pd.concat(gastos)
lista_exclusao_gastos = ['numRessarcimento','dataDocumento','codDocumento', 'codTipoDocumento','numDocumento', 'urlDocumento','codLote', 'parcela']
i = 0

while i < len(lista_exclusao_gastos):
    del total_gastos[lista_exclusao_gastos[i]] 
    i += 1
    
total_gastos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32517 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ano                32517 non-null  float64
 1   mes                32517 non-null  float64
 2   tipoDespesa        32517 non-null  object 
 3   tipoDocumento      32517 non-null  object 
 4   valorDocumento     32517 non-null  float64
 5   nomeFornecedor     32517 non-null  object 
 6   cnpjCpfFornecedor  32517 non-null  object 
 7   valorLiquido       32517 non-null  float64
 8   valorGlosa         32517 non-null  float64
 9   id                 32517 non-null  int64  
dtypes: float64(5), int64(1), object(4)
memory usage: 2.2+ MB


### TRATAMENTO DOS DADOS 
Modelagem dos dados da requisição 'Lista de Gastos'. <hr>

<strong>Informações detalhadas sobre a lista de gastos:<strong> <br> 

In [7]:
total_gastos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32517 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ano                32517 non-null  float64
 1   mes                32517 non-null  float64
 2   tipoDespesa        32517 non-null  object 
 3   tipoDocumento      32517 non-null  object 
 4   valorDocumento     32517 non-null  float64
 5   nomeFornecedor     32517 non-null  object 
 6   cnpjCpfFornecedor  32517 non-null  object 
 7   valorLiquido       32517 non-null  float64
 8   valorGlosa         32517 non-null  float64
 9   id                 32517 non-null  int64  
dtypes: float64(5), int64(1), object(4)
memory usage: 2.2+ MB


Observações:
- Aqui já conseguimos ver se existe diferença entre o count das colunas, sendo possível verificar em quais existem valores nulos. 
- Neste caso não houve a necessidade de tratamento para o tipo dos dados, pois, todos estão retornando de forma correta: <br> Textos em Object/String e Valores calculáveis em Float.

<strong>Verificação e correção dos dados nulos ou duplicados: <strong> <br>

In [8]:
print('LISTAGEM DE COLUNAS COM VALORES NULOS:')
print(total_gastos.isnull().sum())
print('TOTAL DE DADOS VS COLUNAS:')
print(total_gastos.shape)
print('TOTAL DE DADOS VS COLUNAS SEM DUPLICAÇÕES:')
total_gastos.drop_duplicates()
print(total_gastos.shape)

LISTAGEM DE COLUNAS COM VALORES NULOS:
ano                  0
mes                  0
tipoDespesa          0
tipoDocumento        0
valorDocumento       0
nomeFornecedor       0
cnpjCpfFornecedor    0
valorLiquido         0
valorGlosa           0
id                   0
dtype: int64
TOTAL DE DADOS VS COLUNAS:
(32517, 10)
TOTAL DE DADOS VS COLUNAS SEM DUPLICAÇÕES:
(32517, 10)


Observações:
- Neste caso não houve necessidade de ajustes pois não há nenhum dado nulo ou duplicado. <br> 
É possível identificar através da contagem de valores nulos = 0 e também na diferença na contagem dos dados vs colunas com ou sem duplicações.

<strong>Analisando a coluna Fornecedores:<strong> <br> 

In [9]:
print('DADOS FORNECEDORES SEM ALTERAÇÃO:')
print(total_gastos.nomeFornecedor.value_counts().head(15))
print('----------------------------------------------')
print('DADOS FORNECEDORES AJUSTADOS:')
total_gastos.nomeFornecedor = [str(fornecedor).upper().replace("S.A.", "S.A").replace("S/A","S.A").replace("LTDA-ME","LTDA") for fornecedor in total_gastos.nomeFornecedor]
print(total_gastos.nomeFornecedor.value_counts().head(15))

DADOS FORNECEDORES SEM ALTERAÇÃO:
RAMAL                                               1452
CASCOL COMBUSTIVEIS PARA VEICULOS LTDA               883
CORREIOS - SEDEX CONVENCIONAL                        820
Cia Aérea - TAM                                      764
CELULAR FUNCIONAL                                    605
Telefônica Brasil S.A. VIVO                          573
UBER DO BRASIL TECNOLOGIA LTDA.                      488
CORREIOS - CARTA COMERCIAL                           342
CORREIOS - ENCOMENDA PAC                             321
Claro S.A                                            306
SERVICO NACIONAL DE APRENDIZAGEM COMERCIAL SENAC     260
FRATELLI POSTO DE COMBUSTIVEIS LTDA                  231
WMS COMERCIO DE ARTIGOS DE PAPELARIA LTDA-ME         178
SINPETAXI                                            173
CORREIOS - SEDEX 10                                  154
Name: nomeFornecedor, dtype: int64
----------------------------------------------
DADOS FORNECEDORES AJUSTADOS:

Observações:
- Identifiquei que o dado não está padronizado, há alguns com escrita em caixa alta e outros não. Também contém variações de escrita para o mesmo fornecedor.

<strong>Analisando a coluna Tipos de Despesas:<strong> <br> 

In [10]:
print('DADOS FORNECEDORES SEM ALTERAÇÃO:')
print(total_gastos.tipoDespesa.value_counts().head(20))
print('----------------------------------------------')
print('DADOS FORNECEDORES AJUSTADOS:')
total_gastos.tipoDespesa = [str(despesas).upper() for despesas in total_gastos.tipoDespesa]
print(total_gastos.tipoDespesa.value_counts().head(20))

DADOS FORNECEDORES SEM ALTERAÇÃO:
COMBUSTÍVEIS E LUBRIFICANTES.                                11570
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR     5189
TELEFONIA                                                     3836
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                          2543
SERVIÇOS POSTAIS                                              2525
SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                     1729
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                    1293
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES                 1078
Emissão Bilhete Aéreo                                         1001
HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.         611
CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.                  476
PASSAGENS AÉREAS                                               311
SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.       133
ASSINATURA DE PUBLICAÇÕES                                      119
PASSAGENS TERRESTRES, MARÍTI

Observações:
- Identifiquei que nessa coluna o dado também não está padronizado contém alguns com escrita em caixa alta e outros não. 

<strong>Analisando a colunas ano e mês:<strong> <br> 

In [11]:
total_gastos.ano = [str(ano_list).replace("2020.0", "2020") for ano_list in total_gastos.ano]
total_gastos.mes = [str(mes_list).replace("1.0", "1").replace("2.0", "2").replace("3.0", "3").replace("4.0", "4") for mes_list in total_gastos.mes]

Observações:
- Nessa variável fiz um ajuste para 'string' e retirei as casas decimais. 

Para os demais dados da requisição despesas não houve necessidade de ajustes:
- valorDocumento
- cnpjCpfFornecedor
- valorLiquido
- valorGlosa
- id

<strong>Renomeando o nome das colunas:<strong> <br> 

In [12]:
deputados.columns = ['ID', 'NOME', 'PARTIDO','UF','E-MAIL']
deputados.head()

,ID,NOME,PARTIDO,UF,E-MAIL
0,204554,ABÍLIO SANTANA,PL,BA,DEP.ABILIOSANTANA@CAMARA.LEG.BR
1,204521,ABOU ANNI,PSL,SP,DEP.ABOUANNI@CAMARA.LEG.BR
2,204379,ACÁCIO FAVACHO,PROS,AP,DEP.ACACIOFAVACHO@CAMARA.LEG.BR
3,204560,ADOLFO VIANA,PSDB,BA,DEP.ADOLFOVIANA@CAMARA.LEG.BR
4,204528,ADRIANA VENTURA,NOVO,SP,DEP.ADRIANAVENTURA@CAMARA.LEG.BR


In [13]:
total_gastos.columns = ['ANO', 'MÊS', 'TIPO-DESPESA','TIPO-DOCUMENTO','VALOR-DOCUMENTO','NOME-FORNECEDOR','CNPJ-FORNECEDOR','VALOR-LÍQUIDO','VALOR-GLOSA','ID']
total_gastos.head()

,ANO,MÊS,TIPO-DESPESA,TIPO-DOCUMENTO,VALOR-DOCUMENTO,NOME-FORNECEDOR,CNPJ-FORNECEDOR,VALOR-LÍQUIDO,VALOR-GLOSA,ID
0,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,132.76,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,118.35,14.41,204554
1,2020,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,162.27,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,159.97,2.30,204554
2,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1500.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1500.00,0.00,204554
3,2020,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1600.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1600.00,0.00,204554
4,2020,3,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1600.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1600.00,0.00,204554


### CRIANDO UMA BASE COM OS DADOS AJUSTADOS
Criação de um banco de dados SQLite que salva todos os dados já tratados para análises sem a necessidade de requisições na API.
<hr>

In [15]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///dados_abertos.db')
deputados.to_sql('deputados', engine)
total_gastos.to_sql('gastos', engine)

In [16]:
deputados = pd.read_sql("select * from deputados;", engine)
deputados.head()

,index,ID,NOME,PARTIDO,UF,E-MAIL
0,0,204554,ABÍLIO SANTANA,PL,BA,DEP.ABILIOSANTANA@CAMARA.LEG.BR
1,1,204521,ABOU ANNI,PSL,SP,DEP.ABOUANNI@CAMARA.LEG.BR
2,2,204379,ACÁCIO FAVACHO,PROS,AP,DEP.ACACIOFAVACHO@CAMARA.LEG.BR
3,3,204560,ADOLFO VIANA,PSDB,BA,DEP.ADOLFOVIANA@CAMARA.LEG.BR
4,4,204528,ADRIANA VENTURA,NOVO,SP,DEP.ADRIANAVENTURA@CAMARA.LEG.BR


In [18]:
total_gastos = pd.read_sql("select * from gastos;", engine)
total_gastos.head()

,index,ANO,MÊS,TIPO-DESPESA,TIPO-DOCUMENTO,VALOR-DOCUMENTO,NOME-FORNECEDOR,CNPJ-FORNECEDOR,VALOR-LÍQUIDO,VALOR-GLOSA,ID
0,0,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,132.76,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,118.35,14.41,204554
1,1,2020,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Nota Fiscal,162.27,COMPANHIA DE ELETRICIDADE DO ESTADO DA BAHIA,15139629000194,159.97,2.30,204554
2,2,2020,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1500.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1500.00,0.00,204554
3,3,2020,2,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1600.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1600.00,0.00,204554
4,4,2020,3,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,Recibos/Outros,1600.00,MATHEUS QUADROS LOIOLA MARTINS,06540941528,1600.00,0.00,204554
